In [822]:
#train multiclass classification models
#@created by: younis3

In [67]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import tensorflowjs as tfjs

In [68]:
print(tf.__version__)

2.4.0


In [69]:
import sys
print("Python version")
print (sys.version)

Python version
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [70]:
print(tfjs.__version__)

2.8.3


In [71]:
IMG_WIDTH=224
IMG_HEIGHT=224
img_folder=r'C:/Users/younis3/Desktop/V6/Model/train/train_shapes/seg_train/'

In [72]:

# Define constants
IMG_HEIGHT = 80
IMG_WIDTH  = 80
CHANNELS   = 3
batch_size = 8

#train_path = 'C:/Users/younis3/Desktop/train_intel/archive/seg_train/seg_train/'
#test_path  = 'C:/Users/younis3/Desktop/train_intel/archive/seg_test/seg_test/'
#class_names = ['cat', 'chicken', 'duck', 'sheep', 'k','kk']

train_path = 'C:/Users/younis3/Desktop/V6/Model/train/train_shapes/seg_train/'
test_path  = 'C:/Users/younis3/Desktop/V6/Model/train/train_shapes/seg_test/'
#class_names = ['bird', 'bull', 'cat', 'chicken', 'cow', 'dog', 'duck', 'elephant', 'fox', 'horse', 'kangaroo', 'lion', 'penguin', 'sheep']
#class_names = ['banana', 'berry', 'cherry', 'grapes', 'kiwi', 'lemon', 'orange', 'pear', 'pineapple', 'watermelon']
#class_names = ['black', 'blue', 'green', 'orange', 'pink', 'purple', 'red', 'white', 'yellow']
class_names = ['circle', 'diamond', 'heart', 'square', 'star', 'triangle']

In [73]:
## Generators ##
 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

# Training
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)

# Validation
validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation'
)

# Evaluation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size
)

Found 346 images belonging to 6 classes.
Found 36 images belonging to 6 classes.
Found 90 images belonging to 6 classes.


In [74]:
# Build Model
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))


model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(128, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))



model.add(Flatten())
model.add(Dense(6, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 78, 78, 32)        896       
_________________________________________________________________
batch_normalization_20 (Batc (None, 78, 78, 32)        128       
_________________________________________________________________
activation_20 (Activation)   (None, 78, 78, 32)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 37, 37, 32)        9248      
_________________________________________________________________
batch_normalization_21 (Batc (None, 37, 37, 32)        128       
_________________________________________________________________
activation_21 (Activation)   (None, 37, 37, 32)       

In [75]:
# Compile model
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [76]:
#Callbacks
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# lr_redux = ReduceLROnPlateau(monitor='val_loss',
#                             patience = 3, verbose = 1,
#                             factor = 0.1, min_lr = 0.000001)

# log_dir = f"/home/tasos/logs/{int(time.time())}"

# tensorboard = TensorBoard(log_dir=log_dir)
# callbacks = [early_stop, lr_redux, tensorboard]

In [77]:
# Fit model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
    #callbacks=callbacks
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)

print("Test accuracy: {}" .format(test_acc))
print("Test loss: {}" .format(test_loss))

Epoch 1/50
44/44 [==============================] - 5s 102ms/step - loss: 1.1381 - accuracy: 0.6215 - val_loss: 1.7115 - val_accuracy: 0.2222
Epoch 2/50
44/44 [==============================] - 3s 76ms/step - loss: 0.1362 - accuracy: 0.9521 - val_loss: 2.1666 - val_accuracy: 0.3333
Epoch 3/50
44/44 [==============================] - 3s 79ms/step - loss: 0.1458 - accuracy: 0.9624 - val_loss: 2.3815 - val_accuracy: 0.4722
Epoch 4/50
44/44 [==============================] - 3s 72ms/step - loss: 0.0462 - accuracy: 0.9915 - val_loss: 2.3344 - val_accuracy: 0.2500
Epoch 5/50
44/44 [==============================] - 3s 73ms/step - loss: 0.0864 - accuracy: 0.9686 - val_loss: 2.2371 - val_accuracy: 0.5000
Epoch 6/50
44/44 [==============================] - 3s 74ms/step - loss: 0.0286 - accuracy: 0.9941 - val_loss: 1.8483 - val_accuracy: 0.6111
Epoch 7/50
44/44 [==============================] - 3s 72ms/step - loss: 0.0370 - accuracy: 0.9846 - val_loss: 1.7863 - val_accuracy: 0.4722
Epoch 8/50
4

In [78]:
tfjs_target_dir = "C:/Users/younis3/Desktop/V6/Model/savedmodels/shapes"

model.save('model-shapes.model', tfjs_target_dir)
tfjs.converters.save_keras_model(model, tfjs_target_dir)


INFO:tensorflow:Assets written to: model-shapes.model\assets


D:\Apps\Anaconda\lib\site-packages\tensorflowjs\converters\keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [41]:
test_images = []
img_path = 'C:/Users/younis3/Desktop/V6/Model/train/train_fruits/seg_train/dog/dog-!-8.jpg'
img = np.array(PIL.Image.open(img_path))
img = np.resize(img, (IMG_HEIGHT, IMG_WIDTH, CHANNELS))
img = img.astype('float32')
img /= 255

test_images.append(img)
test_images = np.array(test_images)
#plt.imshow(test_images)

model.predict(test_images)
model.predict_classes(test_images)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/younis3/Desktop/V6/Model/train/train_fruits/seg_train/dog/dog-!-8.jpg'

In [837]:
  
#    def plot_image(i, predictions_array, true_label, img):
#     true_label, img = true_label[i], img[i]
#     plt.grid(False)
#     plt.xticks([])
#     plt.yticks([])

#     plt.imshow(img)

#     predicted_label = np.argmax(predictions_array)
#     if predicted_label == true_label:
#         color = 'blue'
#     else:
#         color = 'red'

#     plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
#                             100*np.max(predictions_array),
#                             class_names[true_label]),
#                             color=color)

# def plot_value_array(i, predictions_array, true_label):
#     true_label = true_label[i]
#     plt.grid(False)
#     plt.xticks(range(6))
#     plt.yticks([])
#     thisplot = plt.bar(range(6), predictions_array, color="#777777")
#     plt.ylim([0, 1])
#     predicted_label = np.argmax(predictions_array)

#     thisplot[predicted_label].set_color('red')

